# Vizualizace

Uděláme několik grafů výsledků voleb do Poslanecké sněmovny roků 1996–2017.

In [2]:
import math
import operator
import pandas as pd

import plotly.offline as offline
import plotly.graph_objs as go

from plotly import tools

Načtení csv souborů.

In [3]:
indices = ['year', 'region', 'county', 'town']

In [4]:
df = pd.read_csv('combined.csv', index_col=indices)
votes = pd.read_csv('votes.csv', index_col=indices)

# Souhrnné výsledky voleb

In [5]:
party_yr_sums = df.groupby('year').sum()
vote_yr_sums = votes.groupby('year').sum()

Funkce k získání poměru získaných hlasů strany k celkovému počtu hlasů.

In [6]:
def get_ratio(row):
    votes = vote_yr_sums.loc[row.name]['votes_ok']
    
    def ratio(x):
        return round(x / votes * 100)
    return row.apply(ratio)

Získání souhrnných výsledků voleb z let 1996–2017 ve formě procent.

In [7]:
ratios = party_yr_sums.apply(get_ratio, axis=1)

todrop = [
    c for c in ratios.columns
    if all([x < 5 for x in ratios[c].values if not pd.isna(x)])
]

mask = ratios.apply(lambda x: x.name not in todrop)
ratios.drop(todrop, axis=1, inplace=True)

Vykreslení základního grafu (na linku pak proběhnou úpravy detailů).

In [8]:
def plot(parties):
    data = []
    for c in parties.columns:
        data.append(
            go.Scatter(
                x=parties[c].index,
                y=parties[c].values,
                name=parties[c].name,
                mode='lines+markers',
            ))

    layout = go.Layout(
        xaxis=dict(
            autotick=False,
            ticks="",
            tickvals=parties.index,
            ticktext=[str(y) for y in parties.index]),
        yaxis=dict(autotick=True, ticks='outside'))

    return go.Figure(data=data, layout=layout)


# py.iplot(plot(ratios))

# Výsledky voleb podle krajů

In [9]:
party_reg_sums = df.groupby(['region', 'year']).sum()
vote_reg_sums = votes.groupby(['region', 'year']).sum()

In [10]:
def ratio(n, votes):
    return round(n / votes * 100, 2) if not pd.isna(n) else 0


def get_ratio_reg(row):
    votes = row['votes_ok']
    del row['votes_ok']
    del row['voters']
    del row['votes_all']
    return row.apply(lambda x: ratio(x, votes))


res_and_votes = party_reg_sums.join(vote_reg_sums, how='inner')

ratios = res_and_votes.apply(get_ratio_reg, axis=1)

Funkce k získání *n* nejlépe umístěných stran z daného kraje/v daném roce.

In [11]:
def n_best_all_regs_abs(year, n=3):
    result = {}
    for reg in res_and_votes.index.get_level_values('region').unique():
        best = n_best_reg_abs(n, region=reg)
        if year in best.index:
            result[reg] = {n: r for n, r in best.loc[year].values}
    return result


def n_best_reg_abs(n, region='Hlavní město Praha'):
    per_yr = res_and_votes.loc[region]
    del per_yr['voters']
    del per_yr['votes_all']
    del per_yr['votes_ok']
    return per_yr.apply(lambda r: n_best_yr(n, r), axis=1)


def n_best_all_regs(year, n=3):
    result = {}
    for reg in ratios.index.get_level_values('region').unique():
        best = n_best_reg(n, region=reg)
        if year in best.index:
            result[reg] = {n: r for n, r in best.loc[year].values}
    return result


def n_best_all_yr(year, n=3):
    result = {}
    for reg in ratios.index.get_level_values('region').unique():
        best = n_best_reg(n, region=reg)
        if year in best.index:
            for name, res in best.loc[year].values:
                result.setdefault(name, dict())[reg] = res
    return result


def n_best_reg(n, region='Hlavní město Praha'):
    per_yr = ratios.loc[region]
    return per_yr.apply(lambda r: n_best_yr(n, r), axis=1)


def n_best_yr(n, row):
    top_series = row.sort_values(ascending=False).head(n)
    return pd.Series(list(top_series.items()), range(1, n + 1))

Funkce k vykreslení grafu srovnávajícího nejlepších *n* stran v daných krajích za daný rok.

In [12]:
def plot_best(year, n=3):
    top = n_best_all_regs(year, n=n)
    fig = tools.make_subplots(
        rows=4,
        cols=4,
        shared_yaxes=True,
        subplot_titles=[
            n for n, _ in sorted(top.items(), key=lambda tu: tu[0])
        ])

    j = 0
    for name, parties in sorted(top.items(), key=lambda tu: tu[0]):
        tr = go.Bar(
            x=list(parties.keys()),
            y=list(parties.values()),
            name=name,
            marker=dict(color=[
                colors.get(k, 'rgb(247,247,247)') for k in parties.keys()
            ]),
            hoverinfo='y')
        fig.append_trace(tr, math.ceil((j + 1) / 4), j % 4 + 1)
        j += 1

    fig['layout'].update(
        height=900,
        width=900,
        title=f'Nejlepších 5 stran v jednotlivých krajích v roce {year}',
        showlegend=False)

    for i in range(1, 5):
        key = 'yaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(ticksuffix='%', range=[0, 41])

    for i in range(1, 17):
        key = 'xaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(tickangle=45)

    return fig

In [13]:
colors = {
    'ANO': 'rgb(28,20,94)',
    'KDU': 'rgb(254,223,83)',
    'KDU+USDEU': 'rgb(254,223,83)',
    'KSČM': 'rgb(139,5,11)',
    '(STA)N': 'rgb(146,182,58)',
    'ODA': 'rgb(44,154,107)',
    'ODS': 'rgb(46,176,228)',
    'Piráti': 'rgb(150,206,204)',
    'REPUBLIKÁNI': 'rgb(195,218,242)',
    'SPD': 'rgb(167,103,13)',
    'SZ': 'rgb(44,154,107)',
    'TOP': 'rgb(102,96,167)',
    'US': 'rgb(161,103,165)',
    'VV': 'rgb(33,145,178)',
    'ČSSD': 'rgb(239,149,72)'
}

In [14]:
offline.plot(plot_best(2010, n=5))

This is the format of your plot grid:
[ (1,1) x1,y1 ]   [ (1,2) x2,y1 ]   [ (1,3) x3,y1 ]   [ (1,4) x4,y1 ] 
[ (2,1) x5,y2 ]   [ (2,2) x6,y2 ]   [ (2,3) x7,y2 ]   [ (2,4) x8,y2 ] 
[ (3,1) x9,y3 ]   [ (3,2) x10,y3 ]  [ (3,3) x11,y3 ]  [ (3,4) x12,y3 ]
[ (4,1) x13,y4 ]  [ (4,2) x14,y4 ]  [ (4,3) x15,y4 ]  [ (4,4) x16,y4 ]



'file:///Users/Eugen/Documents/Programování/python/volby/temp-plot.html'

# Zobrazení výsledku v krajích s mandáty

In [15]:
REGIONS = [
    "Hlavní město Praha", "Středočeský kraj", "Jihočeský kraj",
    "Plzeňský kraj", "Karlovarský kraj", "Ústecký kraj", "Liberecký kraj",
    "Královéhradecký kraj", "Pardubický kraj", "Kraj Vysočina",
    "Jihomoravský kraj", "Olomoucký kraj", "Zlínský kraj",
    "Moravskoslezský kraj"
]

mandates = pd.read_csv("mandaty.csv", index_col=['year', 'region'])
mandates.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>KSČM</th>
      <th>ČSSD</th>
      <th>VV</th>
      <th>(STA)N</th>
      <th>KDU</th>
      <th>SZ</th>
      <th>ODS</th>
      <th>Piráti</th>
      <th>Úsvit</th>
      <th>ANO</th>
      <th>TOP</th>
      <th>SPD</th>
    </tr>
    <tr>
      <th>year</th>
      <th>region</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">2006</th>
      <th>Hlavní město Praha</th>
      <td>2</td>
      <td>6</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>1.0</td>
      <td>2.0</td>
      <td>14</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Středočeský kraj</th>
      <td>3</td>
      <td>8</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>10</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Jihočeský kraj</th>
      <td>2</td>
      <td>4</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>1.0</td>
      <td>0.0</td>
      <td>6</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Plzeňský kraj</th>
      <td>2</td>
      <td>4</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>5</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Karlovarský kraj</th>
      <td>1</td>
      <td>2</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>2</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>
</div>

In [16]:
def parties_with_mandates(mds, region):
    return {p:int(res) for p, res in mds.loc[region].items() if not pd.isna(res)}

In [17]:
def plot_best2(year, n=3):
    md_year = mandates.loc[year]
    mds = {reg: parties_with_mandates(md_year, reg) for reg in REGIONS}
    top = n_best_all_regs(year, n=n)
    fig = tools.make_subplots(
        rows=4,
        cols=4,
        shared_yaxes=True,
        subplot_titles=[
            n for n, _ in sorted(top.items(), key=lambda tu: tu[0])
        ])

    j = 0
    for name, parties in sorted(n_best_all_regs(year, n=14).items(), key=lambda tu: tu[0]):
        prt_sorted = sorted(parties.items(), key=lambda tu: tu[1], reverse=True)
        xy = sorted(mds[name].items(), key=lambda tu: operator.indexOf([t[0] for t in prt_sorted], tu[0]))
        tr = go.Bar(
            x=[k for k, _ in xy],
            y=[v for _, v in xy],
            name=name,
            marker=dict(color=[
                colors.get(k, 'rgb(247,247,247)') for k in [k for k, _ in xy]
            ]),
            hoverinfo='y')
        fig.append_trace(tr, math.ceil((j + 1) / 4), j % 4 + 1)
        j += 1

    fig['layout'].update(
        height=900,
        width=900,
        title=f'Rozdělení mandátů v jednotlivých krajích v roce {year}',
        showlegend=False)

    for i in range(1, 5):
        key = 'yaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(range=[0, 11], dtick=2)
    
    for i in range(1, 17):
        key = 'xaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(tickangle=45)

    return fig

In [18]:
offline.plot(plot_best2(2017))

This is the format of your plot grid:
[ (1,1) x1,y1 ]   [ (1,2) x2,y1 ]   [ (1,3) x3,y1 ]   [ (1,4) x4,y1 ] 
[ (2,1) x5,y2 ]   [ (2,2) x6,y2 ]   [ (2,3) x7,y2 ]   [ (2,4) x8,y2 ] 
[ (3,1) x9,y3 ]   [ (3,2) x10,y3 ]  [ (3,3) x11,y3 ]  [ (3,4) x12,y3 ]
[ (4,1) x13,y4 ]  [ (4,2) x14,y4 ]  [ (4,3) x15,y4 ]  [ (4,4) x16,y4 ]



'file:///Users/Eugen/Documents/Programování/python/volby/temp-plot.html'

In [19]:
def plot_best3(year, n=3, rel=True):
    top = n_best_all_regs(year, n=n) if rel else n_best_all_regs_abs(year, n=n)
    fig = tools.make_subplots(
        rows=4,
        cols=4,
        shared_yaxes=True,
        subplot_titles=[
            n for n, _ in sorted(top.items(), key=lambda tu: tu[0])
        ])

    j = 0
    for name, parties in sorted(top.items(), key=lambda tu: tu[0]):
        tr = go.Bar(
            x=list(parties.keys()),
            y=list(parties.values()),
            name=name,
            marker=dict(color=[
                colors.get(k, 'rgb(247,247,247)') for k in parties.keys()
            ]),
            hoverinfo='y')
        fig.append_trace(tr, math.ceil((j + 1) / 4), j % 4 + 1)
        j += 1

    fig['layout'].update(
        height=1400,
        width=1400,
        title=f'Rozdělení mandátů v jednotlivých krajích v roce {year}',
        showlegend=False)

    md_year = mandates.loc[year]
    mds = {reg: parties_with_mandates(md_year, reg) for reg in REGIONS}

    j = 0
    for name, parties in sorted(
            n_best_all_regs(year, n=14).items(), key=lambda tu: tu[0]):
        prt_sorted = sorted(
            parties.items(), key=lambda tu: tu[1], reverse=True)
        xy = sorted(
            mds[name].items(),
            key=lambda tu: operator.indexOf([t[0] for t in prt_sorted], tu[0]))
        tr = go.Scatter(
            x=[k for k, _ in xy],
            y=[v for _, v in xy],
            #width = [0.4 for _ in xy],
            name=name,
            marker=dict(
                color='rgb(255, 102, 153)',
                size=10,
                line=dict(width=2, color='black')),
            hoverinfo='y',
            line=dict(color='black', width=3))
        fig.append_trace(tr, math.ceil((j + 1) / 4), j % 4 + 1)
        j += 1

    for data in fig['data'][len(top.items()):]:
        number = int(data['yaxis'][1:])
        data.update(yaxis=f"y{number+4}")

    for i in range(1, 5):
        key = 'yaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(
                ticksuffix='%' if rel else '',
                range=[0, 41 if rel else 220000],
                showgrid=False)

    for i in range(1, 17):
        key = 'xaxis' + str(i)
        if key in fig['layout']:
            fig['layout'][key].update(tickangle=45)
            fig['layout'][f"yaxis{math.ceil(i / 4) + 4}"] = dict(
                range=[0, 10.5],
                overlaying=f"y{math.ceil(i / 4)}",
                anchor=f"x{i}" if i < 13 else "x14",
                side='right',
                showgrid=True,
                dtick=1,
                title='Počet mandátů',
            )

    return fig

In [20]:
offline.plot(plot_best3(2017, 9, rel=False))

This is the format of your plot grid:
[ (1,1) x1,y1 ]   [ (1,2) x2,y1 ]   [ (1,3) x3,y1 ]   [ (1,4) x4,y1 ] 
[ (2,1) x5,y2 ]   [ (2,2) x6,y2 ]   [ (2,3) x7,y2 ]   [ (2,4) x8,y2 ] 
[ (3,1) x9,y3 ]   [ (3,2) x10,y3 ]  [ (3,3) x11,y3 ]  [ (3,4) x12,y3 ]
[ (4,1) x13,y4 ]  [ (4,2) x14,y4 ]  [ (4,3) x15,y4 ]  [ (4,4) x16,y4 ]



'file:///Users/Eugen/Documents/Programování/python/volby/temp-plot.html'

In [21]:
fig['layout'][f"yaxis{math.ceil(i / 4) + 4}"] = dict(range = [0, 11], 
                          overlaying = f"y{math.ceil(i / 4)}", 
                          anchor = f"x{i}", 
                          side = 'right', 
                          showgrid = False, 
                          title = 'Počet mandátů',
                         )

yaxis=f"{(j + 1) / 4 + 4}"

    for data in fig['data'][len(top.items()):]:
        number = int(data['yaxis'][1:])
        data.update(yaxis=f"y{number+4}")

IndentationError: unexpected indent (<ipython-input-21-acd92a6ff9a8>, line 11)